# [KV Secret Engine](https://developer.hashicorp.com/vault/tutorials/secrets-management/versioned-kv)

In [ ]:
%env  WORKDIR=/tmp/vault

In [ ]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

In [ ]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.0buy0tW2QcQARwJGQsEyg8Ad
%env VAULT_CACERT=/tmp/vault/vault.ca
# %env VAULT_NAMESPACE=admin #Enable if using HCP

In [ ]:
%%bash
vault secrets list -detailed
vault secrets disable secret

In [ ]:
%%bash
#Enable KVv2
vault secrets enable -version=2 -path=secret kv

## Test

In [ ]:
%%bash
## Write secret via CLI to Ops path
vault kv put secret/ops/acme customer_name="ACME Inc." \
        contact_email="john.smith@acme.com"
vault kv put secret/ops/acme2 customer_name="ACME Inc." \
        contact_email="john.smith@acme.com"
vault kv put secret/ops/acme3 customer_name="ACME Inc." \
        contact_email="john.smith@acme.com"


In [ ]:
%%bash
## Write secret via CLI to Dev path
vault kv put secret/dev/acme1 customer_name="Vault Inc." \
        contact_email="jenny.smith@vault.com"
vault kv put secret/dev/acme2 customer_name="Vault Inc." \
        contact_email="jenny.smith@vault.com"
vault kv put secret/dev/acme3 customer_name="Vault Inc." \
        contact_email="jenny.smith@vault.com"

In [ ]:
%%bash 
## Write secret via API
## Include a new version of the secret
cat > $WORKDIR/secret.json <<EOF
{
  "data": {
    "name": "ACME Inc.",
    "contact_email": "jose@acme.com"
  }
}
EOF

## Unlike the kv put, here we have to set the api to v1 and go to the data endpoint
## The result is a new version of the existing secret
curl -k --header "X-Vault-Token: $VAULT_TOKEN" \
   --request POST \
   --data @$WORKDIR/secret.json \
   $VAULT_ADDR/v1/secret/data/ops/acme | jq


In [ ]:
%%bash
## We can add custom metadata
tee $WORKDIR/payload_metadata.json <<EOF
{
  "custom_metadata": {
    "Membership": "Platinum",
    "Region": "US West"
  }
}
EOF
## Request goes to the metadata and not data endpoint
curl -k --header "X-Vault-Token: $VAULT_TOKEN" \
   --request POST \
   --data @$WORKDIR/payload_metadata.json \
   $VAULT_ADDR/v1/secret/metadata/ops/acme
